In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

/Users/kuoteng/.pyenv/versions/3.6.4/envs/NCKU-DSAI/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
DATA_SIZE = 60000
TRAIN_SIZE = 50000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
EPOCH_SIZE = 3
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '-',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

# Data Generation

In [7]:
%time
questions = []
expected = []
seen = set()
print('Generating data...')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs
Generating data...


In [8]:
%time
while len(questions) < DATA_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    if len(questions) % 2 == 0:
        q = '{}-{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a - b)
    else:
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs
Total addition questions: 60000


In [9]:
print(questions[:5], expected[:5])

['202-356', '19+86  ', '71-21  ', '444+817', '49-22  '] ['-154', '105 ', '50  ', '1261', '27  ']


# Processing

In [10]:
print('Vectorization... (to the one-hot encoding)')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization... (to the one-hot encoding)


In [11]:
indices = np.arange(len(y))
np.random.shuffle(indices)
print(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:TRAIN_SIZE]
train_y = y[:TRAIN_SIZE]
test_x = x[TRAIN_SIZE:]
test_y = y[TRAIN_SIZE:]

print('Training Data:')
print(train_x.shape)
print(train_y.shape)

split_at = len(train_x) - len(train_x) // 10
print('split_at', split_at)
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)


[37278  9879  8822 ...  6640 10023 14385]
Training Data:
(50000, 7, 13)
(50000, 4, 13)
split_at 45000
Training Data:
(45000, 7, 13)
(45000, 4, 13)
Validation Data:
(5000, 7, 13)
(5000, 4, 13)
Testing Data:
(10000, 7, 13)
(10000, 4, 13)


In [12]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False  True False False False
   False False]
  [False False False False False  True False False False False False
   False False]
  [False False False False False False False False  True False False
   False False]
  [False False  True False False False False False False False False
   False False]
  [False False False False False False False False False  True False
   False False]
  [False False False False False False False False False False False
   False  True]
  [False False False False  True False False False False False False
   False False]]

 [[False False False False False False False  True False False False
   False False]
  [False False False  True False False False False False False False
   False False]
  [False False False False False False False False False False False
   False  True]
  [False  True False False False False False False False False False
   False False]
  [False False False False False False False False Fals

# Build Model

In [13]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 13)             1677      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 13)             0         
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


# Training

In [14]:
% time
for loop in range(100):
    print()
    print('-' * 50)
    print('Train Loop Num:', loop)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=EPOCH_SIZE,
              validation_data=(x_val, y_val),
              shuffle=True)
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs

--------------------------------------------------
Train Loop Num: 0
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 12s 274us/step - loss: 1.9594 - acc: 0.3056 - val_loss: 1.7946 - val_acc: 0.3400
Epoch 2/3
45000/45000 [==============================] - 10s 224us/step - loss: 1.7557 - acc: 0.3496 - val_loss: 1.7250 - val_acc: 0.3537
Epoch 3/3
45000/45000 [==============================] - 10s 228us/step - loss: 1.6823 - acc: 0.3753 - val_loss: 1.6387 - val_acc: 0.3896
Q 22-213  T -191 ☒ -228
Q 19+251  T 270  ☒ 324 
Q 950+2   T 952  ☒ 992 
Q 254-81  T 173  ☒ 22  
Q 994+1   T 995  ☒ 992 
Q 29+487  T 516  ☒ 508 
Q 679+1   T 680  ☒ 992 
Q 763-54  T 709  ☒ 544 
Q 783-15  T 768  ☒ 764 
Q 584-55  T 529  ☒ 444 

--------------------------------------------------
Train Loop Num: 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [======================

Epoch 3/3
45000/45000 [==============================] - 11s 237us/step - loss: 0.6872 - acc: 0.7433 - val_loss: 0.7086 - val_acc: 0.7261
Q 2+208   T 210  ☒ 213 
Q 50+814  T 864  ☒ 855 
Q 166-30  T 136  ☒ 135 
Q 354+344 T 698  ☒ 699 
Q 295-278 T 17   ☒ 11  
Q 895-224 T 671  ☒ 664 
Q 183+76  T 259  ☒ 251 
Q 110+44  T 154  ☒ 155 
Q 802-383 T 419  ☒ 411 
Q 25-30   T -5   ☑ -5  

--------------------------------------------------
Train Loop Num: 10
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 11s 239us/step - loss: 0.6426 - acc: 0.7602 - val_loss: 0.6801 - val_acc: 0.7387
Epoch 2/3
45000/45000 [==============================] - 13s 284us/step - loss: 0.5983 - acc: 0.7776 - val_loss: 0.6152 - val_acc: 0.7622
Epoch 3/3
45000/45000 [==============================] - 12s 276us/step - loss: 0.5537 - acc: 0.7958 - val_loss: 0.5821 - val_acc: 0.7740
Q 867+20  T 887  ☑ 887 
Q 6-916   T -910 ☒ -918
Q 394+81  T 475  ☒ 466 
Q 106-144 T -38 

45000/45000 [==============================] - 12s 278us/step - loss: 0.0990 - acc: 0.9733 - val_loss: 0.1544 - val_acc: 0.9481
Epoch 2/3
45000/45000 [==============================] - 14s 303us/step - loss: 0.0977 - acc: 0.9734 - val_loss: 0.2338 - val_acc: 0.9222
Epoch 3/3
45000/45000 [==============================] - 11s 244us/step - loss: 0.0973 - acc: 0.9726 - val_loss: 0.1724 - val_acc: 0.9367
Q 0+742   T 742  ☑ 742 
Q 98+56   T 154  ☑ 154 
Q 815+896 T 1711 ☑ 1711
Q 71+163  T 234  ☑ 234 
Q 152-54  T 98   ☒ 10  
Q 56-778  T -722 ☑ -722
Q 514-5   T 509  ☑ 509 
Q 89-705  T -616 ☑ -616
Q 498+30  T 528  ☑ 528 
Q 471+84  T 555  ☑ 555 

--------------------------------------------------
Train Loop Num: 20
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 15s 336us/step - loss: 0.0925 - acc: 0.9745 - val_loss: 0.1633 - val_acc: 0.9414
Epoch 2/3
45000/45000 [==============================] - 10s 233us/step - loss: 0.0924 - acc: 0.97

45000/45000 [==============================] - 13s 287us/step - loss: 0.0855 - acc: 0.9730 - val_loss: 0.1135 - val_acc: 0.9610
Epoch 2/3
45000/45000 [==============================] - 13s 285us/step - loss: 0.0563 - acc: 0.9845 - val_loss: 0.1225 - val_acc: 0.9571
Epoch 3/3
45000/45000 [==============================] - 11s 245us/step - loss: 0.0345 - acc: 0.9918 - val_loss: 0.0932 - val_acc: 0.9677
Q 48+690  T 738  ☑ 738 
Q 787+138 T 925  ☑ 925 
Q 547+337 T 884  ☑ 884 
Q 823-63  T 760  ☑ 760 
Q 72-269  T -197 ☑ -197
Q 2+989   T 991  ☑ 991 
Q 749+846 T 1595 ☒ 1695
Q 211+387 T 598  ☑ 598 
Q 614+3   T 617  ☑ 617 
Q 766-7   T 759  ☒ 769 

--------------------------------------------------
Train Loop Num: 30
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 12s 265us/step - loss: 0.0207 - acc: 0.9971 - val_loss: 0.0903 - val_acc: 0.9681
Epoch 2/3
45000/45000 [==============================] - 11s 252us/step - loss: 0.0199 - acc: 0.99

45000/45000 [==============================] - 11s 238us/step - loss: 0.0244 - acc: 0.9944 - val_loss: 0.0804 - val_acc: 0.9723
Epoch 2/3
45000/45000 [==============================] - 12s 256us/step - loss: 0.0193 - acc: 0.9962 - val_loss: 0.1126 - val_acc: 0.9627
Epoch 3/3
45000/45000 [==============================] - 11s 245us/step - loss: 0.0356 - acc: 0.9912 - val_loss: 0.1011 - val_acc: 0.9666
Q 403+169 T 572  ☑ 572 
Q 562+674 T 1236 ☑ 1236
Q 3+410   T 413  ☑ 413 
Q 254+449 T 703  ☒ 603 
Q 69-508  T -439 ☑ -439
Q 775+1   T 776  ☑ 776 
Q 89+315  T 404  ☑ 404 
Q 3-883   T -880 ☑ -880
Q 85+23   T 108  ☑ 108 
Q 29+354  T 383  ☑ 383 

--------------------------------------------------
Train Loop Num: 40
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 13s 296us/step - loss: 0.0143 - acc: 0.9977 - val_loss: 0.0814 - val_acc: 0.9721
Epoch 2/3
45000/45000 [==============================] - 13s 289us/step - loss: 0.0086 - acc: 0.99

...

45000/45000 [==============================] - 12s 258us/step - loss: 0.0030 - acc: 1.0000 - val_loss: 0.0714 - val_acc: 0.9782
Epoch 2/3
45000/45000 [==============================] - 9s 204us/step - loss: 0.0028 - acc: 1.0000 - val_loss: 0.0710 - val_acc: 0.9782
Epoch 3/3
45000/45000 [==============================] - 9s 202us/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0721 - val_acc: 0.9777
Q 14+372  T 386  ☑ 386 
Q 964+9   T 973  ☑ 973 
Q 92-189  T -97  ☒ -98 
Q 2-163   T -161 ☑ -161
Q 63+315  T 378  ☑ 378 
Q 39-810  T -771 ☑ -771
Q 56+876  T 932  ☑ 932 
Q 416+58  T 474  ☑ 474 
Q 397+782 T 1179 ☑ 1179
Q 7-272   T -265 ☑ -265

--------------------------------------------------
Train Loop Num: 60
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 9s 201us/step - loss: 0.0027 - acc: 1.0000 - val_loss: 0.0783 - val_acc: 0.9754
Epoch 2/3
45000/45000 [==============================] - 9s 201us/step - loss: 0.0982 - acc: 0.9713 -

45000/45000 [==============================] - 10s 213us/step - loss: 0.1020 - acc: 0.9697 - val_loss: 0.1282 - val_acc: 0.9615
Epoch 2/3
45000/45000 [==============================] - 9s 198us/step - loss: 0.0154 - acc: 0.9961 - val_loss: 0.0925 - val_acc: 0.9713
Epoch 3/3
45000/45000 [==============================] - 9s 200us/step - loss: 0.0050 - acc: 0.9995 - val_loss: 0.0791 - val_acc: 0.9749
Q 109-560 T -451 ☑ -451
Q 665+29  T 694  ☑ 694 
Q 982-28  T 954  ☑ 954 
Q 8+21    T 29   ☑ 29  
Q 155+794 T 949  ☑ 949 
Q 384-459 T -75  ☑ -75 
Q 74+11   T 85   ☑ 85  
Q 34-202  T -168 ☑ -168
Q 21+670  T 691  ☑ 691 
Q 121+81  T 202  ☑ 202 

--------------------------------------------------
Train Loop Num: 70
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 9s 199us/step - loss: 0.0029 - acc: 1.0000 - val_loss: 0.0744 - val_acc: 0.9778
Epoch 2/3
45000/45000 [==============================] - 9s 202us/step - loss: 0.0023 - acc: 1.0000 -

45000/45000 [==============================] - 12s 274us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0743 - val_acc: 0.9780
Epoch 2/3
45000/45000 [==============================] - 11s 240us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0740 - val_acc: 0.9784
Epoch 3/3
45000/45000 [==============================] - 11s 239us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0845 - val_acc: 0.9752
Q 879+746 T 1625 ☑ 1625
Q 58+795  T 853  ☑ 853 
Q 574+289 T 863  ☑ 863 
Q 5-626   T -621 ☑ -621
Q 476+28  T 504  ☑ 504 
Q 70+573  T 643  ☑ 643 
Q 945+69  T 1014 ☑ 1014
Q 56+957  T 1013 ☑ 1013
Q 879+746 T 1625 ☑ 1625
Q 91-284  T -193 ☑ -193

--------------------------------------------------
Train Loop Num: 80
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 11s 248us/step - loss: 0.1092 - acc: 0.9691 - val_loss: 0.1081 - val_acc: 0.9668
Epoch 2/3
45000/45000 [==============================] - 11s 241us/step - loss: 0.0098 - acc: 0.99

45000/45000 [==============================] - 11s 246us/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0717 - val_acc: 0.9786
Epoch 2/3
45000/45000 [==============================] - 11s 235us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0715 - val_acc: 0.9786
Epoch 3/3
45000/45000 [==============================] - 11s 251us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0718 - val_acc: 0.9785
Q 417+7   T 424  ☑ 424 
Q 953+9   T 962  ☑ 962 
Q 2-365   T -363 ☑ -363
Q 2-79    T -77  ☑ -77 
Q 979-223 T 756  ☑ 756 
Q 326+159 T 485  ☑ 485 
Q 28-389  T -361 ☑ -361
Q 110+932 T 1042 ☒ 104 
Q 27-116  T -89  ☑ -89 
Q 204-97  T 107  ☑ 107 

--------------------------------------------------
Train Loop Num: 90
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 12s 270us/step - loss: 0.0075 - acc: 0.9982 - val_loss: 0.2549 - val_acc: 0.9317
Epoch 2/3
45000/45000 [==============================] - 10s 230us/step - loss: 0.0826 - acc: 0.97

Q 30-68   T -38  ☑ -38 
Q 65+40   T 105  ☑ 105 
Q 383+66  T 449  ☑ 449 
Q 653+17  T 670  ☑ 670 
Q 2-820   T -818 ☑ -818
Q 947-40  T 907  ☑ 907 
Q 92-552  T -460 ☑ -460
Q 990-56  T 934  ☑ 934 
Q 82+609  T 691  ☑ 691 
Q 974+35  T 1009 ☑ 1009

--------------------------------------------------
Train Loop Num: 99
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
45000/45000 [==============================] - 12s 274us/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.0760 - val_acc: 0.9769
Epoch 2/3
45000/45000 [==============================] - 12s 256us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0743 - val_acc: 0.9777
Epoch 3/3
45000/45000 [==============================] - 11s 237us/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0734 - val_acc: 0.9782
Q 952-78  T 874  ☑ 874 
Q 445+702 T 1147 ☒ 1157
Q 14+44   T 58   ☑ 58  
Q 21+891  T 912  ☒ 902 
Q 662-903 T -241 ☑ -241
Q 61-472  T -411 ☑ -411
Q 165-87  T 78   ☑ 78  
Q 53-55   T -2   ☑ -2  
Q 43-742  T -699 ☑ -699
Q 911-47  

# Testing

In [17]:
print("MSG : Prediction")
print("-" * 50)
right = 0
preds = model.predict_classes(test_x, verbose=0)
for i in range(len(preds)):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))

MSG : Prediction
--------------------------------------------------
Q 6+397   T 403  ☑ 403 
Q 247+68  T 315  ☑ 315 
Q 997+25  T 1022 ☑ 1022
Q 340-5   T 335  ☑ 335 
Q 180-68  T 112  ☑ 112 
Q 494-339 T 155  ☑ 155 
Q 69+311  T 380  ☑ 380 
Q 84-941  T -857 ☑ -857
Q 264-693 T -429 ☒ -439
Q 899-55  T 844  ☑ 844 
Q 0+219   T 219  ☑ 219 
Q 236-33  T 203  ☑ 203 
Q 325+637 T 962  ☑ 962 
Q 202-683 T -481 ☑ -481
Q 8+581   T 589  ☑ 589 
Q 578-80  T 498  ☑ 498 
Q 85-67   T 18   ☑ 18  
Q 237-308 T -71  ☑ -71 
Q 106+1   T 107  ☑ 107 
Q 673+47  T 720  ☑ 720 
Q 622+568 T 1190 ☑ 1190
Q 468+257 T 725  ☑ 725 
Q 543+365 T 908  ☑ 908 
Q 607+62  T 669  ☒ 679 
Q 208-994 T -786 ☒ -787
Q 664+64  T 728  ☑ 728 
Q 70+128  T 198  ☑ 198 
Q 768+2   T 770  ☑ 770 
Q 698-1   T 697  ☑ 697 
Q 446-17  T 429  ☑ 429 
Q 436-53  T 383  ☑ 383 
Q 84+317  T 401  ☑ 401 
Q 691-2   T 689  ☑ 689 
Q 760-342 T 418  ☑ 418 
Q 34+424  T 458  ☑ 458 
Q 56-90   T -34  ☑ -34 
Q 741-87  T 654  ☑ 654 
Q 89-928  T -839 ☑ -839
Q 14-18   T -4   ☑ -

Q 964-954 T 10   ☒ 8   
Q 1-202   T -201 ☑ -201
Q 179-24  T 155  ☑ 155 
Q 265-130 T 135  ☑ 135 
Q 553+801 T 1354 ☑ 1354
Q 66+707  T 773  ☑ 773 
Q 111+98  T 209  ☑ 209 
Q 199+381 T 580  ☒ 570 
Q 36-683  T -647 ☑ -647
Q 256-65  T 191  ☑ 191 
Q 677-41  T 636  ☑ 636 
Q 77-772  T -695 ☑ -695
Q 482+939 T 1421 ☑ 1421
Q 35-371  T -336 ☑ -336
Q 18-859  T -841 ☑ -841
Q 0-282   T -282 ☑ -282
Q 66-297  T -231 ☑ -231
Q 625+34  T 659  ☒ 669 
Q 985-762 T 223  ☑ 223 
Q 5-199   T -194 ☒ -184
Q 264+260 T 524  ☑ 524 
Q 390+8   T 398  ☑ 398 
Q 978+240 T 1218 ☑ 1218
Q 907+221 T 1128 ☑ 1128
Q 50+924  T 974  ☑ 974 
Q 22-301  T -279 ☑ -279
Q 403+50  T 453  ☑ 453 
Q 954+58  T 1012 ☑ 1012
Q 843+92  T 935  ☑ 935 
Q 896-728 T 168  ☑ 168 
Q 109+73  T 182  ☑ 182 
Q 756-124 T 632  ☑ 632 
Q 470+512 T 982  ☑ 982 
Q 51-240  T -189 ☑ -189
Q 63+497  T 560  ☑ 560 
Q 582-49  T 533  ☑ 533 
Q 623-175 T 448  ☑ 448 
Q 677-17  T 660  ☑ 660 
Q 2-10    T -8   ☑ -8  
Q 673-391 T 282  ☑ 282 
Q 75-283  T -208 ☑ -208
Q 45-473  T -428

Q 613+901 T 1514 ☑ 1514
Q 8-996   T -988 ☑ -988
Q 42+50   T 92   ☑ 92  
Q 239-65  T 174  ☑ 174 
Q 1+912   T 913  ☑ 913 
Q 201-782 T -581 ☑ -581
Q 474-84  T 390  ☑ 390 
Q 414-97  T 317  ☑ 317 
Q 17-17   T 0    ☑ 0   
Q 441-123 T 318  ☑ 318 
Q 608+790 T 1398 ☒ 1498
Q 71+589  T 660  ☑ 660 
Q 433+34  T 467  ☑ 467 
Q 77-108  T -31  ☑ -31 
Q 967-15  T 952  ☑ 952 
Q 99-807  T -708 ☑ -708
Q 362+1   T 363  ☑ 363 
Q 719+5   T 724  ☑ 724 
Q 275+78  T 353  ☑ 353 
Q 12+989  T 1001 ☒ 1011
Q 607+17  T 624  ☑ 624 
Q 698-12  T 686  ☑ 686 
Q 934-306 T 628  ☑ 628 
Q 784-20  T 764  ☑ 764 
Q 870+811 T 1681 ☑ 1681
Q 340+56  T 396  ☑ 396 
Q 576-9   T 567  ☑ 567 
Q 605+38  T 643  ☑ 643 
Q 77-505  T -428 ☑ -428
Q 66+135  T 201  ☑ 201 
Q 11-413  T -402 ☑ -402
Q 91+158  T 249  ☒ 259 
Q 593-3   T 590  ☑ 590 
Q 266-253 T 13   ☒ 23  
Q 38-55   T -17  ☑ -17 
Q 54-622  T -568 ☑ -568
Q 47-292  T -245 ☑ -245
Q 320+484 T 804  ☑ 804 
Q 4-388   T -384 ☑ -384
Q 723+677 T 1400 ☑ 1400
Q 308+35  T 343  ☑ 343 
Q 230+45  T 275 

Q 522-983 T -461 ☑ -461
Q 9-419   T -410 ☑ -410
Q 817-618 T 199  ☒ 299 
Q 156-62  T 94   ☒ 96  
Q 329+65  T 394  ☑ 394 
Q 62+28   T 90   ☑ 90  
Q 913-640 T 273  ☑ 273 
Q 747-287 T 460  ☑ 460 
Q 804+52  T 856  ☑ 856 
Q 80-572  T -492 ☑ -492
Q 8-235   T -227 ☑ -227
Q 442+917 T 1359 ☑ 1359
Q 693-548 T 145  ☑ 145 
Q 569+819 T 1388 ☑ 1388
Q 994+479 T 1473 ☑ 1473
Q 89+310  T 399  ☑ 399 
Q 971-998 T -27  ☒ -37 
Q 99+705  T 804  ☑ 804 
Q 724-838 T -114 ☑ -114
Q 569-46  T 523  ☑ 523 
Q 0-960   T -960 ☑ -960
Q 524-14  T 510  ☒ 500 
Q 98+209  T 307  ☑ 307 
Q 9+217   T 226  ☑ 226 
Q 150+659 T 809  ☑ 809 
Q 781-1   T 780  ☑ 780 
Q 45-927  T -882 ☑ -882
Q 22+965  T 987  ☑ 987 
Q 520-797 T -277 ☑ -277
Q 91-685  T -594 ☑ -594
Q 32-794  T -762 ☑ -762
Q 60-422  T -362 ☑ -362
Q 61+613  T 674  ☑ 674 
Q 579+785 T 1364 ☑ 1364
Q 773+53  T 826  ☑ 826 
Q 514+28  T 542  ☑ 542 
Q 527+29  T 556  ☑ 556 
Q 599+912 T 1511 ☑ 1511
Q 86+622  T 708  ☑ 708 
Q 51-461  T -410 ☑ -410
Q 24-76   T -52  ☑ -52 
Q 56-513  T -457

Q 645-672 T -27  ☑ -27 
Q 24+522  T 546  ☑ 546 
Q 95-881  T -786 ☑ -786
Q 761-121 T 640  ☑ 640 
Q 900-40  T 860  ☒ 850 
Q 904-71  T 833  ☒ 823 
Q 194-697 T -503 ☑ -503
Q 641+455 T 1096 ☑ 1096
Q 107-90  T 17   ☒ 27  
Q 0-276   T -276 ☑ -276
Q 447+39  T 486  ☑ 486 
Q 97+481  T 578  ☑ 578 
Q 246+49  T 295  ☑ 295 
Q 58+471  T 529  ☑ 529 
Q 58-732  T -674 ☑ -674
Q 863-61  T 802  ☒ 702 
Q 627-83  T 544  ☑ 544 
Q 918+65  T 983  ☑ 983 
Q 284-413 T -129 ☑ -129
Q 285+847 T 1132 ☑ 1132
Q 5-906   T -901 ☑ -901
Q 623+710 T 1333 ☑ 1333
Q 12-39   T -27  ☑ -27 
Q 6-166   T -160 ☑ -160
Q 916+774 T 1690 ☑ 1690
Q 94+671  T 765  ☑ 765 
Q 18-724  T -706 ☑ -706
Q 19+50   T 69   ☑ 69  
Q 53+449  T 502  ☑ 502 
Q 9-197   T -188 ☑ -188
Q 529-107 T 422  ☑ 422 
Q 934-68  T 866  ☑ 866 
Q 538+6   T 544  ☑ 544 
Q 75-681  T -606 ☑ -606
Q 263+887 T 1150 ☑ 1150
Q 629+0   T 629  ☑ 629 
Q 397-125 T 272  ☑ 272 
Q 574-56  T 518  ☑ 518 
Q 221+549 T 770  ☒ 760 
Q 83+49   T 132  ☑ 132 
Q 587-0   T 587  ☑ 587 
Q 102-174 T -72 

...

Q 369-99  T 270  ☑ 270 
Q 835-2   T 833  ☑ 833 
Q 36-54   T -18  ☑ -18 
Q 773-50  T 723  ☑ 723 
Q 0-13    T -13  ☑ -13 
Q 883-20  T 863  ☑ 863 
Q 862-454 T 408  ☑ 408 
Q 0-306   T -306 ☑ -306
Q 412+65  T 477  ☑ 477 
Q 66+666  T 732  ☑ 732 
Q 610-837 T -227 ☒ -237
Q 16+347  T 363  ☒ 364 
Q 388+871 T 1259 ☑ 1259
Q 56+78   T 134  ☑ 134 
Q 22-506  T -484 ☑ -484
Q 80-7    T 73   ☑ 73  
Q 53+523  T 576  ☑ 576 
Q 650-972 T -322 ☑ -322
Q 21+28   T 49   ☑ 49  
Q 309-8   T 301  ☑ 301 
Q 964+87  T 1051 ☑ 1051
Q 877+991 T 1868 ☑ 1868
Q 62+902  T 964  ☑ 964 
Q 327+18  T 345  ☑ 345 
Q 884+862 T 1746 ☑ 1746
Q 398-360 T 38   ☒ 28  
Q 42+814  T 856  ☑ 856 
Q 493+5   T 498  ☑ 498 
Q 70-30   T 40   ☑ 40  
Q 917-86  T 831  ☑ 831 
Q 94+817  T 911  ☑ 911 
Q 668-37  T 631  ☑ 631 
Q 653-31  T 622  ☑ 622 
Q 1+452   T 453  ☑ 453 
Q 0-983   T -983 ☑ -983
Q 89-534  T -445 ☑ -445
Q 470-95  T 375  ☑ 375 
Q 619-452 T 167  ☑ 167 
Q 5+404   T 409  ☑ 409 
Q 94-501  T -407 ☑ -407
Q 636-25  T 611  ☑ 611 
Q 87-546  T -459

Q 877+612 T 1489 ☑ 1489
Q 68-807  T -739 ☑ -739
Q 204+78  T 282  ☑ 282 
Q 887-159 T 728  ☑ 728 
Q 120-853 T -733 ☑ -733
Q 51+918  T 969  ☒ 979 
Q 42+672  T 714  ☑ 714 
Q 41-660  T -619 ☑ -619
Q 213+3   T 216  ☑ 216 
Q 501-11  T 490  ☑ 490 
Q 436+12  T 448  ☑ 448 
Q 80-529  T -449 ☑ -449
Q 96-727  T -631 ☑ -631
Q 776-407 T 369  ☑ 369 
Q 51-608  T -557 ☑ -557
Q 723+291 T 1014 ☑ 1014
Q 59-547  T -488 ☑ -488
Q 810-207 T 603  ☑ 603 
Q 1+489   T 490  ☑ 490 
Q 822+2   T 824  ☑ 824 
Q 213-294 T -81  ☑ -81 
Q 9-584   T -575 ☑ -575
Q 43-9    T 34   ☑ 34  
Q 5-402   T -397 ☑ -397
Q 65-721  T -656 ☑ -656
Q 835+1   T 836  ☑ 836 
Q 48+974  T 1022 ☑ 1022
Q 31-821  T -790 ☑ -790
Q 51-10   T 41   ☑ 41  
Q 552+3   T 555  ☑ 555 
Q 57-635  T -578 ☑ -578
Q 466+12  T 478  ☑ 478 
Q 621+53  T 674  ☑ 674 
Q 10-67   T -57  ☑ -57 
Q 643-209 T 434  ☑ 434 
Q 63+96   T 159  ☑ 159 
Q 87+391  T 478  ☑ 478 
Q 363+872 T 1235 ☑ 1235
Q 118+48  T 166  ☑ 166 
Q 885-448 T 437  ☑ 437 
Q 661-543 T 118  ☑ 118 
Q 9-179   T -170

Q 700-213 T 487  ☑ 487 
Q 125+499 T 624  ☑ 624 
Q 32+197  T 229  ☑ 229 
Q 968-123 T 845  ☑ 845 
Q 375+801 T 1176 ☑ 1176
Q 10+212  T 222  ☑ 222 
Q 5-546   T -541 ☑ -541
Q 897-27  T 870  ☑ 870 
Q 411+25  T 436  ☑ 436 
Q 262+743 T 1005 ☑ 1005
Q 695-0   T 695  ☑ 695 
Q 30+78   T 108  ☑ 108 
Q 300-90  T 210  ☑ 210 
Q 339-326 T 13   ☒ 33  
Q 69-24   T 45   ☑ 45  
Q 781+74  T 855  ☑ 855 
Q 196+803 T 999  ☒ 109 
Q 943-814 T 129  ☑ 129 
Q 67-153  T -86  ☑ -86 
Q 608+705 T 1313 ☑ 1313
Q 912-317 T 595  ☑ 595 
Q 775+237 T 1012 ☑ 1012
Q 632+674 T 1306 ☑ 1306
Q 33+795  T 828  ☑ 828 
Q 8+920   T 928  ☑ 928 
Q 77+378  T 455  ☑ 455 
Q 356+7   T 363  ☑ 363 
Q 534+98  T 632  ☑ 632 
Q 8+318   T 326  ☑ 326 
Q 204-10  T 194  ☑ 194 
Q 3-500   T -497 ☑ -497
Q 71-104  T -33  ☒ -32 
Q 75-454  T -379 ☑ -379
Q 391-100 T 291  ☑ 291 
Q 778+0   T 778  ☑ 778 
Q 4+516   T 520  ☑ 520 
Q 204+7   T 211  ☑ 211 
Q 21+640  T 661  ☑ 661 
Q 564+994 T 1558 ☑ 1558
Q 974+45  T 1019 ☑ 1019
Q 254+3   T 257  ☑ 257 
Q 41+481  T 522 

Q 569-534 T 35   ☑ 35  
Q 147+765 T 912  ☑ 912 
Q 868+82  T 950  ☑ 950 
Q 11+462  T 473  ☑ 473 
Q 778+83  T 861  ☑ 861 
Q 564+37  T 601  ☑ 601 
Q 391-121 T 270  ☑ 270 
Q 877-67  T 810  ☑ 810 
Q 453+38  T 491  ☑ 491 
Q 607-172 T 435  ☑ 435 
Q 2-560   T -558 ☑ -558
Q 716+36  T 752  ☑ 752 
Q 36+42   T 78   ☑ 78  
Q 268-802 T -534 ☑ -534
Q 222+430 T 652  ☑ 652 
Q 445-8   T 437  ☑ 437 
Q 61+976  T 1037 ☑ 1037
Q 24-293  T -269 ☑ -269
Q 848-28  T 820  ☑ 820 
Q 468+16  T 484  ☑ 484 
Q 202-214 T -12  ☒ -8  
Q 0+404   T 404  ☑ 404 
Q 899+428 T 1327 ☑ 1327
Q 114+292 T 406  ☒ 306 
Q 946+5   T 951  ☑ 951 
Q 121-264 T -143 ☑ -143
Q 79-871  T -792 ☑ -792
Q 766-409 T 357  ☑ 357 
Q 424-94  T 330  ☑ 330 
Q 675+51  T 726  ☑ 726 
Q 31-96   T -65  ☑ -65 
Q 199+38  T 237  ☑ 237 
Q 49+613  T 662  ☑ 662 
Q 236+74  T 310  ☒ 300 
Q 13+444  T 457  ☑ 457 
Q 968-688 T 280  ☑ 280 
Q 871+59  T 930  ☒ 920 
Q 96-797  T -701 ☑ -701
Q 100+92  T 192  ☒ 292 
Q 4+686   T 690  ☑ 690 
Q 82+529  T 611  ☑ 611 
Q 897+5   T 902 

Q 671+452 T 1123 ☑ 1123
Q 0-80    T -80  ☑ -80 
Q 8-965   T -957 ☑ -957
Q 77+173  T 250  ☑ 250 
Q 38+233  T 271  ☑ 271 
Q 944-126 T 818  ☑ 818 
Q 863-75  T 788  ☑ 788 
Q 839+4   T 843  ☑ 843 
Q 36+565  T 601  ☑ 601 
Q 84+497  T 581  ☑ 581 
Q 210+638 T 848  ☑ 848 
Q 423-340 T 83   ☑ 83  
Q 6+203   T 209  ☑ 209 
Q 23-861  T -838 ☑ -838
Q 81-874  T -793 ☑ -793
Q 268+80  T 348  ☑ 348 
Q 466-811 T -345 ☑ -345
Q 885+78  T 963  ☑ 963 
Q 544+821 T 1365 ☑ 1365
Q 931+7   T 938  ☑ 938 
Q 74+405  T 479  ☑ 479 
Q 155-814 T -659 ☑ -659
Q 688+13  T 701  ☑ 701 
Q 796-48  T 748  ☑ 748 
Q 24+502  T 526  ☑ 526 
Q 32-671  T -639 ☑ -639
Q 229-454 T -225 ☑ -225
Q 271-272 T -1   ☑ -1  
Q 873-547 T 326  ☑ 326 
Q 439-4   T 435  ☑ 435 
Q 874+18  T 892  ☑ 892 
Q 55+304  T 359  ☑ 359 
Q 296+60  T 356  ☑ 356 
Q 374+44  T 418  ☑ 418 
Q 56+413  T 469  ☑ 469 
Q 519+77  T 596  ☑ 596 
Q 158+909 T 1067 ☑ 1067
Q 92-404  T -312 ☑ -312
Q 785+50  T 835  ☑ 835 
Q 79-450  T -371 ☑ -371
Q 910+33  T 943  ☑ 943 
Q 890-511 T 379 

In [18]:
# print("MSG : Prediction")
# test_x = ["555+275", "860+7  ", "340+29 "]
# test_y = ["830 ", "867 ", "369 "]
# x = np.zeros((len(test_x), MAXLEN, len(chars)), dtype=np.bool)
# y = np.zeros((len(test_y), DIGITS + 1, len(chars)), dtype=np.bool)
# for j, (i, c) in enumerate(zip(test_x, test_y)):
#     x[j] = ctable.encode(i, MAXLEN)
#     y[j] = ctable.encode(c, DIGITS + 1)